In [4]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('./kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


./kaggle/input\chatbot-data\intents.json
./kaggle/input\computer-science-theory-qa-dataset\intents.json


In [6]:
intents1 = pd.read_json('./kaggle/input/chatbot-data/intents.json')

In [8]:
intents1.shape
intents1.head()

,intents
0,"{'tag': 'greeting', 'patterns': ['Hi', 'How ar..."
1,"{'tag': 'goodbye', 'patterns': ['cya', 'see yo..."
2,"{'tag': 'creator', 'patterns': ['what is the n..."
3,"{'tag': 'name', 'patterns': ['name', 'your nam..."
4,"{'tag': 'hours', 'patterns': ['timing of colle..."


In [10]:
intents2 = pd.read_json('./kaggle/input/computer-science-theory-qa-dataset/intents.json')

In [12]:
intents2.shape

(172, 1)

In [14]:
intents2.head()

,intents
0,"{'tag': 'abstraction', 'patterns': ['Explain d..."
1,"{'tag': 'error', 'patterns': ['What is a synta..."
2,"{'tag': 'documentation', 'patterns': ['Explain..."
3,"{'tag': 'testing', 'patterns': ['What is softw..."
4,"{'tag': 'datastructure', 'patterns': ['How do ..."


In [16]:
%%bash
jq -s '[.[].intents] | add | {intents: .}' \
    ./kaggle/input/chatbot-data/intents.json \
    ./kaggle/input/computer-science-theory-qa-dataset/intents.json \
    > merged_intents.json
echo "Done"

Done


In [18]:
import json
import numpy as np
import random
import nltk
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.optimizers import Adam
from nltk.stem import SnowballStemmer

with open('./kaggle/working/merged_intents.json', 'r') as file:
    data = json.load(file)

In [20]:
stemmer = SnowballStemmer("english")

words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])


words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []
out_empty = [0 for _ in range(len(labels))]


sequence_length = 5
for i in range(len(docs_x) - sequence_length):
    seq_in = docs_x[i:i + sequence_length]
    seq_out = docs_y[i + sequence_length]

    bag = []
    for seq in seq_in:
        wrds = [stemmer.stem(w.lower()) for w in seq]
        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(seq_out)] = 1

    training.append(bag)
    output.append(output_row)

training = np.array(training)
training = np.reshape(training, (training.shape[0], sequence_length, len(words)))

output = np.array(output)

In [22]:
model = Sequential()
model.add(LSTM(550, input_shape=(training.shape[1], training.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(labels), activation='softmax'))

model.summary()
model.compile(Adam(learning_rate=.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(training, output, epochs=500, verbose=1, batch_size=4)

C:\Users\halco\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 5, 550)              │       2,362,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 5, 550)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 5, 512)              │       2,177,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 5, 512)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 5, 256)              │         787,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 5, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 5, 128)              │         197,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 5, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 209)                 │          13,585 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,587,393 (21.31 MB)

 Trainable params: 5,587,393 (21.31 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/500
187/187 ━━━━━━━━━━━━━━━━━━━━ 15s 50ms/step - accuracy: 0.0342 - loss: 5.1179
Epoch 2/500
187/187 ━━━━━━━━━━━━━━━━━━━━ 10s 51ms/step - accuracy: 0.0331 - loss: 4.4876
Epoch 3/500
187/187 ━━━━━━━━━━━━━━━━━━━━ 10s 52ms/step - accuracy: 0.0471 - loss: 4.3667
Epoch 4/500
187/187 ━━━━━━━━━━━━━━━━━━━━ 10s 52ms/step - accuracy: 0.0429 - loss: 4.3341
Epoch 5/500
187/187 ━━━━━━━━━━━━━━━━━━━━ 10s 52ms/step - accuracy: 0.0568 - loss: 4.3104
Epoch 6/500
187/187 ━━━━━━━━━━━━━━━━━━━━ 10s 52ms/step - accuracy: 0.0462 - loss: 4.1314
Epoch 7/500
187/187 ━━━━━━━━━━━━━━━━━━━━ 10s 52ms/step - accuracy: 0.0552 - loss: 4.0920
Epoch 8/500
187/187 ━━━━━━━━━━━━━━━━━━━━ 10s 55ms/step - accuracy: 0.0577 - loss: 4.0348
Epoch 9/500
187/187 ━━━━━━━━━━━━━━━━━━━━ 12s 65ms/step - accuracy: 0.1156 - loss: 3.8427
Epoch 10/500
187/187 ━━━━━━━━━━━━━━━━━━━━ 13s 71ms/step - accuracy: 0.0766 - loss: 3.8038
Epoch 11/500
187/187 ━━━━━━━━━━━━━━━━━━━━ 11s 61ms/step - accuracy: 0.1017 - loss: 3.7058
Epoch 12/500
187/18

In [24]:
model.save("chatbot.h5")